###문제 S-3: 성적 합계 및 평균.

##pyspark 기본 셋팅

In [1]:
import os
import pyspark

myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession\
    .builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

##RDD 생성

In [2]:
from pyspark.sql.types import StructField, StructType, StringType, LongType
marks=[
    "'김하나','English', 100",
    "'김하나','Math', 80",
    "'임하나','English', 70",
    "'임하나','Math', 100",
    "'김갑돌','English', 82.3",
    "'김갑돌','Math', 98.5"
]
marksRdd=spark.sparkContext.parallelize(marks)

##3-1번문제 이름 합계

In [3]:
marksByName = marksRdd.map(lambda x:x.split(','))\
    .map(lambda x:(x[0], float(x[2])))

In [4]:
sumByName = marksByName.reduceByKey(lambda x,y: x+y).collect()

In [5]:
for i in sumByName:
    print(i[0], i[1])

'김하나' 180.0
'임하나' 170.0
'김갑돌' 180.8


##3-2번문제 과목 합계

In [6]:
marksBySubject = marksRdd.map(lambda x:x.split(','))\
    .map(lambda x:(x[1], float(x[2])))

In [7]:
sumBySubject = marksBySubject.reduceByKey(lambda x,y: x+y).collect()

In [8]:
for i in sumBySubject:
    print(i[0], i[1])

'English' 252.3
'Math' 278.5


##3-3번문제 합계와 갯수

In [9]:
countByName=marksByName.combineByKey(
    (lambda x: (x,1)),
    (lambda x, value: (x[0]+value, x[1]+1)),
    (lambda x, y: (x[0]+y[0], x[1]+y[1])))

In [10]:
for i in countByName.collect():
    print(i[0], i[1])

'김하나' (180.0, 2)
'임하나' (170.0, 2)
'김갑돌' (180.8, 2)


##3-4번문제 평균

In [11]:
averageByName = countByName\
    .map(lambda x:(x[0],(sum,count)):(key,float(sum)/count))\
    .collect()

In [12]:
for i in averageByName:
    print(i[0], i[1])

'김하나' 90.0
'임하나' 85.0
'김갑돌' 90.4
